In [2]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import sys
import math
import xesmf as xe

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import filter_utils as filt
from ecpaper_utils import averaging_utils as avg

importlib.reload(read)
importlib.reload(cal)
importlib.reload(filt)
importlib.reload(avg)

import warnings
warnings.filterwarnings('ignore')

Set up model names, dates and CLIVAR_LE location

In [3]:
modnames=['CanESM2','CSIRO-Mk3-6-0','CESM1-CAM5','GFDL-CM3','MPI-ESM']
modnames_small=['canesm2','csiro_mk36','cesm','gfdl_cm3','mpi']
basedir="/glade/collections/cdg/data/CLIVAR_LE/"
ybegp=1979 ; monbegp=1 ; yendp=2014 ; monendp=12
ybegf=2070 ; monbegf=1 ; yendf=2099 ; monendf=12
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

nmonthsp=(yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf=(yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

plevuse="30000"

pathout="/glade/scratch/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

Loop over models and members and output the DJF climatology of meridional wind at 300hPa

In [7]:
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

for imod in range(len(modnames)):
    print(modnames[imod])
    moddir=basedir+modnames_small[imod]+"_lens/Amon/va/"
    memfiles = !cd $moddir && ls *rcp85*.nc
    nmems=len(memfiles)
    reusewgt=False
    wgtfile="/glade/scratch/islas/python/tmp/wgtfile.nc"
    
    for imem in range(len(memfiles)):
        fname=memfiles[imem]
        print(fname)
        vpast = read.read_1lev(moddir+fname, datebegp, dateendp, plevuse)
        vfut = read.read_1lev(moddir+fname, datebegf, dateendf, plevuse)
        
        # check sizes
        if (vpast.time.size != nmonthsp):
            print("somethings's wrong, "+str(nmonthspast)+
                  "expected, but got "+str(vpast.time.size))
            sys.exit("failure at "+fname+" past period")
            
        if (vfut.time.size != nmonthsf):
            print("something's wrong, "+str(nmonthsfut)+
                 "expected, but got "+str(vfut.time.size))
            sys.exit("failure at "+fname+" future period")

        vpastdjf_ts = cal.season_ts(vpast,"va", season="DJF")
        vpastdjf=cal.season_mean(vpast, "va", season="DJF")
        vfutdjf=cal.season_mean(vfut, "va", season="DJF")
        
        regridder = xe.Regridder(vpastdjf, grid_out, 'bilinear',
                                periodic=True, reuse_weights=reusewgt,
                                filename=wgtfile)
        reusewgt=True
        vpastinterp = regridder(vpastdjf)
        vfutinterp = regridder(vfutdjf)
        vpastdjf_tsinterp=regridder(vpastdjf_ts)
        
        if (imem == 0):
            vpast_djf = xr.DataArray(np.zeros([nmems, nlat, nlon]),
                                    coords=[np.arange(0,nmems,1), vpastinterp["lat"], vpastinterp["lon"]],
                                            dims=["member", "lat", "lon"], name="vpast_djf")
            vfut_djf = xr.DataArray(np.zeros([nmems, nlat, nlon]),
                                    coords=[np.arange(0,nmems,1), vfutinterp["lat"], vfutinterp["lon"]],
                                            dims=["member", "lat", "lon"], name="vfut_djf")
            vpast_djf_ts = xr.DataArray(np.zeros([nmems, vpastdjf_ts.time.size, nlat, nlon]),
                                        coords=[np.arange(0,nmems,1), vpastdjf_ts.time, vpastdjf_tsinterp["lat"], vpastdjf_tsinterp["lon"]],
                                        dims=["member","time","lat","lon"], name="vpast_djf_ts")
            
        vpast_djf[imem,:,:]=vpastinterp
        vfut_djf[imem,:,:]=vfutinterp
        vpast_djf_ts[imem,:,:,:]=vpastdjf_tsinterp
        
    vpast_djf.to_netcdf(path=pathout+"v300_"+modnames[imod]+".nc")
    vfut_djf.to_netcdf(path=pathout+"v300_"+modnames[imod]+".nc", mode="a")
    vpast_djf_ts.to_netcdf(path=pathout+"v300timeseris_"+modnames[imod]+".nc")
        
        

CanESM2
va_Amon_CanESM2_historical_rcp85_r10i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r11i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r12i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r13i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r14i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r15i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r16i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r17i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r18i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r19i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r1i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r20i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r21i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r22i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r23i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r24i1p1_195001-210012.nc
va_Amon_CanESM2_historical_rcp85_r25i1p1_195001-210012.nc
va_Amon